In [15]:
import pandas as pd
import numpy as np

In [16]:
ds =pd.read_csv("PS_20174392719_1491204439457_log.csv")
print(ds.head())

   step      type    amount     nameOrig  oldbalanceOrg  newbalanceOrig  \
0     1   PAYMENT   9839.64  C1231006815       170136.0       160296.36   
1     1   PAYMENT   1864.28  C1666544295        21249.0        19384.72   
2     1  TRANSFER    181.00  C1305486145          181.0            0.00   
3     1  CASH_OUT    181.00   C840083671          181.0            0.00   
4     1   PAYMENT  11668.14  C2048537720        41554.0        29885.86   

      nameDest  oldbalanceDest  newbalanceDest  isFraud  isFlaggedFraud  
0  M1979787155             0.0             0.0        0               0  
1  M2044282225             0.0             0.0        0               0  
2   C553264065             0.0             0.0        1               0  
3    C38997010         21182.0             0.0        1               0  
4  M1230701703             0.0             0.0        0               0  


In [17]:
# Checking for null values or not
print(ds.isnull().sum())

step              0
type              0
amount            0
nameOrig          0
oldbalanceOrg     0
newbalanceOrig    0
nameDest          0
oldbalanceDest    0
newbalanceDest    0
isFraud           0
isFlaggedFraud    0
dtype: int64


Exploring Transaction Type

In [18]:
print(ds.type.value_counts())

CASH_OUT    2237500
PAYMENT     2151495
CASH_IN     1399284
TRANSFER     532909
DEBIT         41432
Name: type, dtype: int64


In [19]:
type = ds["type"].value_counts()
transactions = type.index
quantity = type.values

In [20]:
!pip install plotly

In [25]:
import plotly.express as px
figure = px.pie(ds, values=quantity, names=transactions, hole = 0.5,
               title = "Distribution of Transaction Type")
figure.show()

Checking Correlation

In [22]:
correlation = ds.corr()
print(correlation["isFraud"].sort_values(ascending=False))

isFraud           1.000000
amount            0.076688
isFlaggedFraud    0.044109
step              0.031578
oldbalanceOrg     0.010154
newbalanceDest    0.000535
oldbalanceDest   -0.005885
newbalanceOrig   -0.008148
Name: isFraud, dtype: float64


In [23]:
# Transforming the categorical features into numerical
# and transforming the values of the isFraud column into No Fraud

ds["type"] = ds["type"].map({"CASH_OUT":1,"PAYMENT":2,
                              "CASH_IN":3, "TRANSFER":4,
                              "DEBIT":5})
ds["isFraud"] = ds['isFraud'].map({0: "No Fraud", 1:'Fraud'})

In [24]:
print(ds)

         step  type      amount     nameOrig  oldbalanceOrg  newbalanceOrig  \
0           1     2     9839.64  C1231006815      170136.00       160296.36   
1           1     2     1864.28  C1666544295       21249.00        19384.72   
2           1     4      181.00  C1305486145         181.00            0.00   
3           1     1      181.00   C840083671         181.00            0.00   
4           1     2    11668.14  C2048537720       41554.00        29885.86   
...       ...   ...         ...          ...            ...             ...   
6362615   743     1   339682.13   C786484425      339682.13            0.00   
6362616   743     4  6311409.28  C1529008245     6311409.28            0.00   
6362617   743     1  6311409.28  C1162922333     6311409.28            0.00   
6362618   743     4   850002.52  C1685995037      850002.52            0.00   
6362619   743     1   850002.52  C1280323807      850002.52            0.00   

            nameDest  oldbalanceDest  newbalanceDes

Splitting the dataset into the training set and test set.
Creating the classification model

In [28]:
x = np.array(ds[['type', 'amount', 'oldbalanceOrg','newbalanceOrig']])
y = np.array(ds[['isFraud']])

In [29]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.10, random_state = 42 )

In [31]:
from sklearn.tree import DecisionTreeClassifier
classifier = DecisionTreeClassifier()
classifier.fit(x_train, y_train)
print(classifier.score(x_test, y_test))

0.9997422445470576


In [32]:
# prediction
# features = [type, amount, oldbalanceOrg, newbalanceOrig]
features = np.array([[4, 9000.60, 9000.60, 0.0]])
print(classifier.predict(features))

['Fraud']


In [35]:
y_pred = classifier.predict(x_test)
print(y_pred)

['No Fraud' 'No Fraud' 'No Fraud' ... 'No Fraud' 'No Fraud' 'No Fraud']


In [36]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test,y_pred)
print(cm)
accuracy_score(y_test,y_pred)

[[   728     89]
 [    75 635370]]


0.9997422445470576